Задание
Создайте приложение для эмуляции работа киоска
по продаже хот-догов. Приложение должно иметь следующую функциональность:

Файл: ingredients.py (Ингредиенты)

In [2]:
# ingredients.py
from uuid import uuid4
from typing import Dict, Any

class Ingredient:
    """
    Представляет ингредиент для хот-дога.
    """
    def __init__(self, name: str, cost_to_kiosk: float, price_to_customer: float, category: str):
        self.id = str(uuid4()) # Уникальный ID для каждого экземпляра ингредиента
        self.name = name # Название ингредиента
        self.cost_to_kiosk = cost_to_kiosk  # Себестоимость для киоска
        self.price_to_customer = price_to_customer # Цена для клиента при добавлении вручную
        self.category = category # Категория (например, "булочка", "сосиска", "соус", "топпинг")

    def __str__(self) -> str:
        return f"{self.name} (Закуп.: ${self.cost_to_kiosk:.2f}, Цена: ${self.price_to_customer:.2f})"

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует объект ингредиента в словарь для сохранения."""
        return {
            "id": self.id,
            "name": self.name,
            "cost_to_kiosk": self.cost_to_kiosk,
            "price_to_customer": self.price_to_customer,
            "category": self.category,
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Ingredient':
        """Создает объект ингредиента из словаря."""
        ingredient = cls(
            name=data["name"],
            cost_to_kiosk=data["cost_to_kiosk"],
            price_to_customer=data["price_to_customer"],
            category=data["category"]
        )
        ingredient.id = data.get("id", ingredient.id) # Используем существующий ID, если есть
        return ingredient

Файл: inventory_system.py (Система инвентаризации и наблюдатели)

In [3]:
# inventory_system.py
import json
import os
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Tuple, Any


class IStockObserver(ABC): 
    """Интерфейс Наблюдателя за Складом."""
    @abstractmethod
    def update_low_stock(self, ingredient_name: str, current_stock: int):
        """Обновляет информацию о низком запасе."""
        pass

class ConsoleLowStockNotifier(IStockObserver): 
    """Консольный Уведомитель о Низком Запасе."""
    def update_low_stock(self, ingredient_name: str, current_stock: int):
        print(f"⚠️ ПРЕДУПРЕЖДЕНИЕ О НИЗКОМ ЗАПАСЕ: '{ingredient_name}' заканчивается. Текущий запас: {current_stock}. Пожалуйста, закажите еще.")

class InventoryManager: 
    """
    Управляет запасом ингредиентов. (Принцип единственной ответственности)
    Реализован как Одиночка (Singleton).
    """
    _instance = None 

    def __new__(cls, *args, **kwargs): 
        if not cls._instance:
            cls._instance = super(InventoryManager, cls).__new__(cls)
        return cls._instance

    def __init__(self, inventory_file: str = "inventory_data.json", low_stock_threshold: int = 5):
        if not hasattr(self, '_initialized'): # Гарантируем, что __init__ выполняется только один раз
            self.inventory_file = inventory_file 
            self._stock: Dict[str, Tuple[Ingredient, int]] = {}  
            self._low_stock_threshold = low_stock_threshold 
            self._observers: List[IStockObserver] = [] 
            self.load_inventory() 
            self._initialized = True

    def add_observer(self, observer: IStockObserver): 
        if observer not in self._observers:
            self._observers.append(observer)

    def remove_observer(self, observer: IStockObserver): 
        self._observers.remove(observer)

    def _notify_low_stock(self, ingredient_name: str, current_stock: int): 
        for observer in self._observers:
            observer.update_low_stock(ingredient_name, current_stock)

    def load_inventory(self): 
        if os.path.exists(self.inventory_file):
            try:
                with open(self.inventory_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for item_data in data:
                        ingredient = Ingredient.from_dict(item_data["ingredient"])
                        self._stock[ingredient.name] = (ingredient, item_data["quantity"])
                print("Инвентарь успешно загружен.")
            except Exception as e:
                print(f"Ошибка загрузки инвентаря из файла '{self.inventory_file}': {e}. Начинаем с пустого инвентаря.")
                self._stock = {}
        else:
            print(f"Файл инвентаря '{self.inventory_file}' не найден. Начинаем с пустого инвентаря.")
            self._stock = {} 

    def save_inventory(self): 
        data_to_save = []
        for ingredient_name, (ingredient_obj, quantity) in self._stock.items():
            data_to_save.append({
                "ingredient": ingredient_obj.to_dict(),
                "quantity": quantity
            })
        try:
            with open(self.inventory_file, 'w', encoding='utf-8') as f:
                json.dump(data_to_save, f, indent=4, ensure_ascii=False)
            # print("Инвентарь сохранен.") # Можно раскомментировать для отладки
        except Exception as e:
            print(f"Ошибка сохранения инвентаря в файл '{self.inventory_file}': {e}")

    def add_ingredient_to_stock(self, ingredient: Ingredient, quantity: int): 
        if ingredient.name in self._stock:
            _, current_qty = self._stock[ingredient.name]
            self._stock[ingredient.name] = (ingredient, current_qty + quantity)
        else:
            self._stock[ingredient.name] = (ingredient, quantity)
        print(f"Добавлено {quantity} шт. ингредиента '{ingredient.name}' на склад. Новое общее количество: {self._stock[ingredient.name][1]}")
        self.save_inventory()

    def get_ingredient_details(self, name: str) -> Optional[Ingredient]: 
        if name in self._stock:
            return self._stock[name][0]
        return None

    def get_stock_quantity(self, name: str) -> int: 
        return self._stock.get(name, (None, 0))[1]

    def use_ingredient(self, name: str, quantity_to_use: int = 1) -> bool: 
        if name in self._stock:
            ingredient_obj, current_qty = self._stock[name]
            if current_qty >= quantity_to_use:
                new_qty = current_qty - quantity_to_use
                self._stock[name] = (ingredient_obj, new_qty)
                self.save_inventory()
                if new_qty <= self._low_stock_threshold:
                    self._notify_low_stock(name, new_qty)
                if new_qty == 0:
                     print(f"🚨 ИНГРЕДИЕНТ ЗАКОНЧИЛСЯ: '{name}' больше нет на складе.")
                return True
        print(f"Ошибка использования: Ингредиент '{name}' отсутствует в достаточном количестве или не существует.")
        return False

    def get_available_ingredients_by_category(self, category: str) -> List[Ingredient]: 
        return [ing for ing, qty in self._stock.values() if ing.category == category and qty > 0]

    def display_inventory(self): 
        print("\n--- Текущий Состав Инвентаря ---")
        if not self._stock:
            print("Инвентарь пуст.")
            return
        for name, (ingredient, quantity) in self._stock.items():
            print(f"- {name}: {quantity} шт. (Закупочная цена: ${ingredient.cost_to_kiosk:.2f}, Цена для клиента: ${ingredient.price_to_customer:.2f})")
        print("---------------------------------")
    
    def get_ingredients_to_purchase(self) -> List[Tuple[str, int]]: 
        to_purchase = []
        for name, (ingredient, quantity) in self._stock.items():
            if quantity <= self._low_stock_threshold:
                to_purchase.append((name, self._low_stock_threshold - quantity + 1)) 
        return to_purchase

Файл: strategies.py (Стратегии скидок и оплаты)

In [4]:
# strategies.py
from abc import ABC, abstractmethod
from typing import List, Any 
# from hotdog_module import HotDog # Если стратегии будут работать с объектами HotDog
# from order_system import OrderItem # Если стратегии будут работать с OrderItem

# --- Паттерн Стратегия для Скидок ---
class IDiscountStrategy(ABC): 
    """Абстрактный интерфейс Стратегии Скидок."""
    @abstractmethod
    def calculate_discount(self, items_in_order: List[Any], current_total_price: float) -> float: 
        """Рассчитывает сумму скидки."""
        pass

class QuantityDiscountStrategy(IDiscountStrategy): 
    """Конкретная Стратегия: скидка за количество хот-догов."""
    def __init__(self, min_quantity: int = 3, discount_percentage: float = 0.10): # 10% скидка
        self.min_quantity = min_quantity
        self.discount_percentage = discount_percentage

    def calculate_discount(self, items_in_order: List[Any], current_total_price: float) -> float:
        if len(items_in_order) >= self.min_quantity:
            discount = current_total_price * self.discount_percentage
            print(f"Применена скидка {self.discount_percentage*100:.0f}% за заказ {len(items_in_order)} хот-догов: -${discount:.2f}")
            return discount
        return 0.0

class NoDiscountStrategy(IDiscountStrategy): 
    """Конкретная Стратегия: без скидки."""
    def calculate_discount(self, items_in_order: List[Any], current_total_price: float) -> float:
        return 0.0

# --- Паттерн Стратегия для Оплаты ---
class IPaymentStrategy(ABC): 
    """Абстрактный интерфейс Стратегии Оплаты."""
    @abstractmethod
    def pay(self, amount: float) -> bool: 
        """Производит оплату."""
        pass

    @abstractmethod
    def get_payment_type_name(self) -> str: 
        """Возвращает название типа оплаты."""
        pass

class CashPaymentStrategy(IPaymentStrategy): 
    """Конкретная Стратегия: оплата наличными."""
    def pay(self, amount: float) -> bool:
        print(f"Обработка оплаты наличными на сумму ${amount:.2f}...")
        # Здесь могла бы быть логика приема наличных, расчета сдачи и т.д.
        print("Оплата наличными прошла успешно.")
        return True
    def get_payment_type_name(self) -> str:
        return "Наличные"

class CardPaymentStrategy(IPaymentStrategy): 
    """Конкретная Стратегия: оплата картой."""
    def pay(self, amount: float) -> bool:
        print(f"Обработка оплаты картой на сумму ${amount:.2f}...")
        # Здесь могла бы быть интеграция с платежным терминалом или API
        card_number = input("Введите номер карты (имитация для примера): ")
        if card_number and len(card_number) > 10: # Простейшая имитация проверки
            print("Оплата картой прошла успешно.")
            return True
        print("Оплата картой не удалась (неверные данные карты или ошибка обработки).")
        return False
    def get_payment_type_name(self) -> str:
        return "Карта"

Файл: hotdog_module.py (Модуль хот-догов: определение, рецепты, строитель)

In [7]:
# hotdog_module.py
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Any


class HotDog: 
    """
    Представляет один хот-дог. (Принцип единственной ответственности)
    """
    def __init__(self, name: str):
        self.name = name # Название хот-дога
        self.ingredients: List[Ingredient] = [] # Список ингредиентов
        self.kiosk_cost: float = 0.0 # Себестоимость для киоска
        self.customer_price: float = 0.0 # Цена для клиента

    def add_ingredient(self, ingredient: Ingredient): 
        self.ingredients.append(ingredient)
        self.kiosk_cost += ingredient.cost_to_kiosk
        self.customer_price += ingredient.price_to_customer # Для кастомных сборок

    def set_fixed_price(self, price: float): 
        self.customer_price = price

    def __str__(self) -> str:
        ingredient_names = ", ".join(sorted([ing.name for ing in self.ingredients]))
        return (f"Хот-дог: {self.name}\n"
                f"  Ингредиенты: {ingredient_names}\n"
                f"  Себестоимость для киоска: ${self.kiosk_cost:.2f}\n"
                f"  Цена для клиента: ${self.customer_price:.2f}")

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует объект хот-дога в словарь."""
        return {
            "name": self.name,
            "ingredients": [ing.to_dict() for ing in self.ingredients],
            "kiosk_cost": self.kiosk_cost,
            "customer_price": self.customer_price,
        }

class HotDogRecipe: 
    """
    Определяет стандартный рецепт хот-дога. (Принцип единственной ответственности)
    """
    def __init__(self, name: str, ingredient_names: List[str], fixed_price: float):
        self.name = name # Название рецепта
        self.ingredient_names = ingredient_names # Список названий ингредиентов
        self.fixed_price = fixed_price # Фиксированная цена по рецепту

# --- Паттерн Строитель (Builder Pattern) ---
class IHotDogBuilder(ABC): 
    """Абстрактный интерфейс Строителя Хот-догов."""
    @abstractmethod
    def reset(self, name: str): 
        pass

    @abstractmethod
    def add_base_ingredient(self, ingredient: Ingredient): 
        pass
    
    @abstractmethod
    def add_sauce(self, ingredient: Ingredient): 
        pass

    @abstractmethod
    def add_topping(self, ingredient: Ingredient): 
        pass
    
    @abstractmethod
    def set_fixed_price(self, price: float): 
        pass

    @abstractmethod
    def get_hotdog(self) -> Optional[HotDog]: 
        pass

class CustomHotDogBuilder(IHotDogBuilder): 
    """Конкретный Строитель для создания кастомных хот-догов."""
    def __init__(self):
        self._hotdog: Optional[HotDog] = None

    def reset(self, name: str = "Кастомный Хот-дог"):
        self._hotdog = HotDog(name)
        return self 

    def add_base_ingredient(self, ingredient: Ingredient):
        if self._hotdog:
            self._hotdog.add_ingredient(ingredient)
        return self

    def add_sauce(self, ingredient: Ingredient):
        if self._hotdog:
            self._hotdog.add_ingredient(ingredient)
        return self

    def add_topping(self, ingredient: Ingredient):
        if self._hotdog:
            self._hotdog.add_ingredient(ingredient)
        return self
    
    def set_fixed_price(self, price: float): 
        if self._hotdog:
            self._hotdog.set_fixed_price(price)
        return self

    def get_hotdog(self) -> Optional[HotDog]:
        product = self._hotdog
        # self.reset() # Можно сбрасывать строитель после получения продукта для повторного использования
        return product

class HotDogDirector: 
    """
    Директор управляет процессом сборки хот-дога с помощью Строителя.
    Используется для сборки по стандартным рецептам.
    """
    def __init__(self, builder: IHotDogBuilder, inventory: InventoryManager):
        self._builder = builder
        self._inventory = inventory

    def build_from_recipe(self, recipe: HotDogRecipe) -> Optional[HotDog]: 
        self._builder.reset(recipe.name)
        all_ingredients_available = True
        temp_ingredients_to_use: Dict[str, int] = {} 

        # Проверяем наличие всех ингредиентов из рецепта на складе
        for ing_name in recipe.ingredient_names:
            temp_ingredients_to_use[ing_name] = temp_ingredients_to_use.get(ing_name, 0) + 1
            if self._inventory.get_stock_quantity(ing_name) < temp_ingredients_to_use[ing_name]:
                print(f"Недостаточно ингредиента '{ing_name}' на складе для рецепта '{recipe.name}'.")
                all_ingredients_available = False
                break
        
        if not all_ingredients_available:
            return None # Не можем собрать хот-дог

        # Если все ингредиенты доступны, добавляем их через строителя
        built_ingredients_for_inventory_update = []
        for ing_name in recipe.ingredient_names:
            ingredient_obj = self._inventory.get_ingredient_details(ing_name)
            if ingredient_obj: 
                self._builder.add_base_ingredient(ingredient_obj) # Используем общий метод добавления
                built_ingredients_for_inventory_update.append(ingredient_obj)
            else: 
                print(f"Критическая ошибка: Детали ингредиента '{ing_name}' не найдены после проверки запасов.")
                return None # Должно быть найдено, если проверка прошла
        
        self._builder.set_fixed_price(recipe.fixed_price)
        hotdog = self._builder.get_hotdog()

        # Если хот-дог успешно создан, списываем ингредиенты со склада
        if hotdog:
            for ing in built_ingredients_for_inventory_update:
                 self._inventory.use_ingredient(ing.name) 
        return hotdog

Файл: order_system.py (Система заказов и их сохранения)

In [9]:
# order_system.py
import json
import os
import datetime
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Any

class OrderItem: 
    """
    Представляет один хот-дог в контексте заказа.
    Хранит информацию о хот-доге на момент продажи.
    """
    def __init__(self, hotdog: HotDog):
        self.hotdog_name = hotdog.name # Название хот-дога
        self.ingredients = [ing.name for ing in hotdog.ingredients] # Список названий ингредиентов
        self.price_paid = hotdog.customer_price # Цена, уплаченная клиентом
        self.cost_to_kiosk = hotdog.kiosk_cost # Себестоимость для киоска

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует элемент заказа в словарь."""
        return self.__dict__
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'OrderItem':
        """Создает элемент заказа из словаря (упрощенная версия)."""
        # В реальном приложении здесь могло бы быть воссоздание объекта HotDog,
        # но для простоты хранения истории используем только данные.
        item = cls(HotDog(data["hotdog_name"])) # Создаем фиктивный HotDog для конструктора
        item.ingredients = data["ingredients"]
        item.price_paid = data["price_paid"]
        item.cost_to_kiosk = data["cost_to_kiosk"]
        return item

class Order: 
    """
    Представляет заказ клиента. (Принцип единственной ответственности)
    """
    def __init__(self, order_id: str):
        self.order_id = order_id # Уникальный ID заказа
        self.items: List[OrderItem] = [] # Список элементов заказа
        self.timestamp = datetime.datetime.now().isoformat() # Временная метка создания заказа
        self.subtotal_price: float = 0.0 # Промежуточная цена без скидки
        self.total_kiosk_cost: float = 0.0 # Общая себестоимость всех хот-догов в заказе
        self.discount_applied: float = 0.0 # Сумма примененной скидки
        self.final_price: float = 0.0 # Итоговая цена к оплате
        self.payment_method: Optional[str] = None # Использованный метод оплаты
        self.profit: float = 0.0 # Прибыль от заказа

    def add_hotdog(self, hotdog: HotDog): 
        self.items.append(OrderItem(hotdog))
        self.subtotal_price += hotdog.customer_price
        self.total_kiosk_cost += hotdog.kiosk_cost

    def apply_discount_and_finalize(self, 
                                    discount_strategy: IDiscountStrategy, 
                                    payment_strategy: IPaymentStrategy) -> bool: 
        """Применяет скидку, обрабатывает оплату и финализирует заказ."""
        self.discount_applied = discount_strategy.calculate_discount(self.items, self.subtotal_price) 
        self.final_price = self.subtotal_price - self.discount_applied
        
        payment_successful = payment_strategy.pay(self.final_price)
        if payment_successful:
            self.payment_method = payment_strategy.get_payment_type_name()
            self.profit = self.final_price - self.total_kiosk_cost
            return True
        return False

    def __str__(self) -> str:
        order_details = f"\n--- Заказ ID: {self.order_id} (Время: {self.timestamp}) ---\n"
        for item in self.items:
            order_details += f"- Хот-дог: {item.hotdog_name} (Цена: ${item.price_paid:.2f})\n"
            order_details += f"  Состав: {', '.join(item.ingredients)}\n"
        order_details += f"Промежуточный итог: ${self.subtotal_price:.2f}\n"
        if self.discount_applied > 0:
            order_details += f"Скидка: -${self.discount_applied:.2f}\n"
        order_details += f"Итоговая цена: ${self.final_price:.2f}\n"
        order_details += f"Метод оплаты: {self.payment_method or 'Не указан'}\n"
        order_details += f"Общая себестоимость для киоска: ${self.total_kiosk_cost:.2f}\n"
        order_details += f"Прибыль по заказу: ${self.profit:.2f}\n"
        return order_details

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует объект заказа в словарь."""
        return {
            "order_id": self.order_id,
            "timestamp": self.timestamp,
            "items": [item.to_dict() for item in self.items],
            "subtotal_price": self.subtotal_price,
            "total_kiosk_cost": self.total_kiosk_cost,
            "discount_applied": self.discount_applied,
            "final_price": self.final_price,
            "payment_method": self.payment_method,
            "profit": self.profit
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Order':
        """Создает объект заказа из словаря."""
        order = cls(data["order_id"])
        order.timestamp = data["timestamp"]
        order.items = [OrderItem.from_dict(item_data) for item_data in data["items"]]
        order.subtotal_price = data["subtotal_price"]
        order.total_kiosk_cost = data["total_kiosk_cost"]
        order.discount_applied = data["discount_applied"]
        order.final_price = data["final_price"]
        order.payment_method = data["payment_method"]
        order.profit = data["profit"]
        return order

# --- Система Сохранения Заказов ---
class IOrderPersistence(ABC): 
    """Абстрактный интерфейс для сохранения и загрузки заказов."""
    @abstractmethod
    def save_order(self, order: Order): 
        pass

    @abstractmethod
    def load_all_orders(self) -> List[Order]: 
        pass

class FileOrderPersistence(IOrderPersistence): 
    """
    Конкретная реализация сохранения заказов в файл (JSON Lines формат).
    (Принцип единственной ответственности)
    """
    def __init__(self, filename: str = "all_orders_history.jsonl"): 
        self.filename = filename

    def save_order(self, order: Order):
        try:
            with open(self.filename, 'a', encoding='utf-8') as f: 
                json.dump(order.to_dict(), f, ensure_ascii=False)
                f.write('\n') 
            print(f"Заказ {order.order_id} успешно сохранен в файл '{self.filename}'.")
        except IOError as e:
            print(f"Ошибка при сохранении заказа {order.order_id} в файл: {e}")

    def load_all_orders(self) -> List[Order]:
        orders: List[Order] = []
        if not os.path.exists(self.filename):
            print(f"Файл истории заказов '{self.filename}' не найден. Возвращен пустой список.")
            return orders
        try:
            with open(self.filename, 'r', encoding='utf-8') as f:
                for line_number, line in enumerate(f, 1):
                    if line.strip(): 
                        try:
                            data = json.loads(line)
                            orders.append(Order.from_dict(data))
                        except json.JSONDecodeError as json_e:
                            print(f"Ошибка декодирования JSON в файле '{self.filename}', строка {line_number}: {json_e}")
                            # Можно пропустить строку или прервать загрузку
        except IOError as e:
            print(f"Ошибка при загрузке заказов из файла '{self.filename}': {e}")
        return orders

Файл: order_system.py (Система заказов и их сохранения)

In [10]:
# order_system.py
import json
import os
import datetime
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Any

class OrderItem: 
    """
    Представляет один хот-дог в контексте заказа.
    Хранит информацию о хот-доге на момент продажи.
    """
    def __init__(self, hotdog: HotDog):
        self.hotdog_name = hotdog.name # Название хот-дога
        self.ingredients = [ing.name for ing in hotdog.ingredients] # Список названий ингредиентов
        self.price_paid = hotdog.customer_price # Цена, уплаченная клиентом
        self.cost_to_kiosk = hotdog.kiosk_cost # Себестоимость для киоска

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует элемент заказа в словарь."""
        return self.__dict__
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'OrderItem':
        """Создает элемент заказа из словаря (упрощенная версия)."""
        # В реальном приложении здесь могло бы быть воссоздание объекта HotDog,
        # но для простоты хранения истории используем только данные.
        item = cls(HotDog(data["hotdog_name"])) # Создаем фиктивный HotDog для конструктора
        item.ingredients = data["ingredients"]
        item.price_paid = data["price_paid"]
        item.cost_to_kiosk = data["cost_to_kiosk"]
        return item

class Order: 
    """
    Представляет заказ клиента. (Принцип единственной ответственности)
    """
    def __init__(self, order_id: str):
        self.order_id = order_id # Уникальный ID заказа
        self.items: List[OrderItem] = [] # Список элементов заказа
        self.timestamp = datetime.datetime.now().isoformat() # Временная метка создания заказа
        self.subtotal_price: float = 0.0 # Промежуточная цена без скидки
        self.total_kiosk_cost: float = 0.0 # Общая себестоимость всех хот-догов в заказе
        self.discount_applied: float = 0.0 # Сумма примененной скидки
        self.final_price: float = 0.0 # Итоговая цена к оплате
        self.payment_method: Optional[str] = None # Использованный метод оплаты
        self.profit: float = 0.0 # Прибыль от заказа

    def add_hotdog(self, hotdog: HotDog): 
        self.items.append(OrderItem(hotdog))
        self.subtotal_price += hotdog.customer_price
        self.total_kiosk_cost += hotdog.kiosk_cost

    def apply_discount_and_finalize(self, 
                                    discount_strategy: IDiscountStrategy, 
                                    payment_strategy: IPaymentStrategy) -> bool: 
        """Применяет скидку, обрабатывает оплату и финализирует заказ."""
        self.discount_applied = discount_strategy.calculate_discount(self.items, self.subtotal_price) 
        self.final_price = self.subtotal_price - self.discount_applied
        
        payment_successful = payment_strategy.pay(self.final_price)
        if payment_successful:
            self.payment_method = payment_strategy.get_payment_type_name()
            self.profit = self.final_price - self.total_kiosk_cost
            return True
        return False

    def __str__(self) -> str:
        order_details = f"\n--- Заказ ID: {self.order_id} (Время: {self.timestamp}) ---\n"
        for item in self.items:
            order_details += f"- Хот-дог: {item.hotdog_name} (Цена: ${item.price_paid:.2f})\n"
            order_details += f"  Состав: {', '.join(item.ingredients)}\n"
        order_details += f"Промежуточный итог: ${self.subtotal_price:.2f}\n"
        if self.discount_applied > 0:
            order_details += f"Скидка: -${self.discount_applied:.2f}\n"
        order_details += f"Итоговая цена: ${self.final_price:.2f}\n"
        order_details += f"Метод оплаты: {self.payment_method or 'Не указан'}\n"
        order_details += f"Общая себестоимость для киоска: ${self.total_kiosk_cost:.2f}\n"
        order_details += f"Прибыль по заказу: ${self.profit:.2f}\n"
        return order_details

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует объект заказа в словарь."""
        return {
            "order_id": self.order_id,
            "timestamp": self.timestamp,
            "items": [item.to_dict() for item in self.items],
            "subtotal_price": self.subtotal_price,
            "total_kiosk_cost": self.total_kiosk_cost,
            "discount_applied": self.discount_applied,
            "final_price": self.final_price,
            "payment_method": self.payment_method,
            "profit": self.profit
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Order':
        """Создает объект заказа из словаря."""
        order = cls(data["order_id"])
        order.timestamp = data["timestamp"]
        order.items = [OrderItem.from_dict(item_data) for item_data in data["items"]]
        order.subtotal_price = data["subtotal_price"]
        order.total_kiosk_cost = data["total_kiosk_cost"]
        order.discount_applied = data["discount_applied"]
        order.final_price = data["final_price"]
        order.payment_method = data["payment_method"]
        order.profit = data["profit"]
        return order

# --- Система Сохранения Заказов ---
class IOrderPersistence(ABC): 
    """Абстрактный интерфейс для сохранения и загрузки заказов."""
    @abstractmethod
    def save_order(self, order: Order): 
        pass

    @abstractmethod
    def load_all_orders(self) -> List[Order]: 
        pass

class FileOrderPersistence(IOrderPersistence): 
    """
    Конкретная реализация сохранения заказов в файл (JSON Lines формат).
    (Принцип единственной ответственности)
    """
    def __init__(self, filename: str = "all_orders_history.jsonl"): 
        self.filename = filename

    def save_order(self, order: Order):
        try:
            with open(self.filename, 'a', encoding='utf-8') as f: 
                json.dump(order.to_dict(), f, ensure_ascii=False)
                f.write('\n') 
            print(f"Заказ {order.order_id} успешно сохранен в файл '{self.filename}'.")
        except IOError as e:
            print(f"Ошибка при сохранении заказа {order.order_id} в файл: {e}")

    def load_all_orders(self) -> List[Order]:
        orders: List[Order] = []
        if not os.path.exists(self.filename):
            print(f"Файл истории заказов '{self.filename}' не найден. Возвращен пустой список.")
            return orders
        try:
            with open(self.filename, 'r', encoding='utf-8') as f:
                for line_number, line in enumerate(f, 1):
                    if line.strip(): 
                        try:
                            data = json.loads(line)
                            orders.append(Order.from_dict(data))
                        except json.JSONDecodeError as json_e:
                            print(f"Ошибка декодирования JSON в файле '{self.filename}', строка {line_number}: {json_e}")
                            # Можно пропустить строку или прервать загрузку
        except IOError as e:
            print(f"Ошибка при загрузке заказов из файла '{self.filename}': {e}")
        return orders

Файл: reporting_module.py (Модуль отчетов)

In [12]:
# reporting_module.py

class SalesReporter: 
    """
    Отвечает за генерацию отчетов о продажах. (Принцип единственной ответственности)
    """
    def __init__(self, order_persistence: IOrderPersistence):
        self.order_persistence = order_persistence

    def generate_report(self): 
        orders = self.order_persistence.load_all_orders()
        if not orders:
            print("Нет данных о заказах для генерации отчета.")
            return

        total_hotdogs_sold = sum(len(order.items) for order in orders)
        total_revenue = sum(order.final_price for order in orders)
        total_profit = sum(order.profit for order in orders)

        print("\n--- Отчет о Продажах Киоска ---")
        print(f"Общее количество проданных хот-догов: {total_hotdogs_sold}")
        print(f"Общая выручка: ${total_revenue:.2f}")
        print(f"Общая прибыль: ${total_profit:.2f}")
        print("---------------------------------")

Файл: kiosk_facade.py (Фасад киоска)

In [15]:
# kiosk_facade.py
from uuid import uuid4
from typing import List, Dict, Optional


class HotDogKiosk: 
    """
    Фасад для управления всеми операциями киоска хот-догов.
    Оркестрирует взаимодействие между различными компонентами системы.
    (Принцип единственной ответственности за оркестрацию)
    Зависит от абстракций. (Принцип инверсии зависимостей)
    """
    def __init__(self, inventory_manager: InventoryManager,
                 hotdog_builder: IHotDogBuilder,
                 order_persistence: IOrderPersistence,
                 discount_strategy: IDiscountStrategy,
                 payment_strategies: Dict[str, IPaymentStrategy],
                 standard_recipes: List[HotDogRecipe]):
        self.inventory = inventory_manager
        self.builder = hotdog_builder
        # Директор использует Строителя и Инвентарь для создания хот-догов по рецептам
        self.director = HotDogDirector(self.builder, self.inventory) 
        self.order_persistence = order_persistence
        self.active_discount_strategy = discount_strategy 
        self.payment_strategies = payment_strategies 
        # Храним стандартные рецепты в словаре для быстрого доступа по имени
        self.standard_recipes: Dict[str, HotDogRecipe] = {recipe.name.lower(): recipe for recipe in standard_recipes}
        self.current_order_hotdogs: List[HotDog] = [] # Хот-доги в текущем формируемом заказе

    def get_available_standard_recipes(self) -> List[HotDogRecipe]: 
        return list(self.standard_recipes.values())
        
    def start_new_order(self): 
        self.current_order_hotdogs = []
        print("\nНачат новый заказ. Корзина очищена.")

    def add_standard_hotdog_to_order(self, recipe_name: str) -> bool: 
        recipe = self.standard_recipes.get(recipe_name.lower())
        if not recipe:
            print(f"Ошибка: Стандартный рецепт '{recipe_name}' не найден.")
            return False
        
        hotdog = self.director.build_from_recipe(recipe) # Директор создает хот-дог
        if hotdog:
            self.current_order_hotdogs.append(hotdog)
            print(f"Хот-дог по рецепту '{hotdog.name}' успешно добавлен в текущий заказ.")
            print(hotdog) # Показываем детали добавленного хот-дога
            return True
        else:
            print(f"Не удалось добавить хот-дог по рецепту '{recipe_name}' (возможно, нет ингредиентов).")
            return False

    def add_custom_hotdog_to_order(self, built_hotdog: HotDog) -> bool: 
        # Предполагается, что 'built_hotdog' уже проверен на наличие ингредиентов
        # и они были списаны на этапе его сборки в UI или другом потоке.
        self.current_order_hotdogs.append(built_hotdog)
        print(f"Кастомный хот-дог '{built_hotdog.name}' успешно добавлен в текущий заказ.")
        print(built_hotdog)
        return True

    def display_current_order(self): 
        if not self.current_order_hotdogs:
            print("Текущий заказ пуст.")
            return
        print("\n--- Содержимое Текущего Заказа ---")
        total_price = 0
        for i, hd in enumerate(self.current_order_hotdogs):
            print(f"{i+1}. {hd.name} - Цена: ${hd.customer_price:.2f}")
            total_price += hd.customer_price
        print(f"Промежуточный итог по заказу: ${total_price:.2f}")
        print("----------------------------------")

    def finalize_and_pay_order(self, payment_method_key: str) -> Optional[Order]: 
        if not self.current_order_hotdogs:
            print("Невозможно завершить пустой заказ.")
            return None

        payment_strategy = self.payment_strategies.get(payment_method_key.lower()) # Приводим к нижнему регистру для надежности
        if not payment_strategy:
            print(f"Ошибка: Выбран неверный метод оплаты '{payment_method_key}'.")
            return None

        order = Order(order_id=str(uuid4())) # Создаем новый объект заказа
        for hd in self.current_order_hotdogs:
            order.add_hotdog(hd) # Добавляем хот-доги из текущей "корзины" в заказ
        
        if order.apply_discount_and_finalize(self.active_discount_strategy, payment_strategy):
            self.order_persistence.save_order(order)
            print("\n--- Заказ Успешно Оформлен и Оплачен ---")
            print(order) # Отображаем детали финального заказа
            self.start_new_order() # Очищаем корзину для следующего клиента
            return order
        else:
            print("Оплата заказа не удалась. Заказ не был финализирован.")
            # В реальной системе здесь могла бы быть логика возврата ингредиентов на склад,
            # если они были списаны предварительно.
            return None

    def view_sales_report(self): 
        # Для генерации отчета создаем экземпляр SalesReporter

        reporter = SalesReporter(self.order_persistence)
        reporter.generate_report()

    def view_inventory(self): 
        self.inventory.display_inventory()

    def view_ingredients_to_purchase(self): 
        self.inventory.view_ingredients_to_purchase()

Файл: ui_console.py (Консольный интерфейс пользователя)

In [17]:
# ui_console.py
from typing import Optional

class ConsoleUI: 
    """Отвечает за взаимодействие с пользователем через консоль."""
    def __init__(self, kiosk: HotDogKiosk, inventory: InventoryManager): 
        self.kiosk = kiosk
        self.inventory = inventory # Нужен для проверки наличия ингредиентов при кастомной сборке

    def run(self): 
        print("Добро пожаловать в автоматизированный киоск по продаже хот-догов!")
        self.kiosk.start_new_order() # Начинаем с пустого заказа

        while True:
            self._print_main_menu()
            choice = input("Введите номер выбранного пункта меню: ").strip()
            if choice == '1':
                self._handle_add_hotdog_to_order()
            elif choice == '2':
                self.kiosk.display_current_order()
            elif choice == '3':
                self._handle_finalize_and_pay_order()
            elif choice == '4':
                self.kiosk.view_inventory()
            elif choice == '5':
                self.kiosk.view_sales_report()
            elif choice == '6':
                self.kiosk.view_ingredients_to_purchase()
            elif choice == '0':
                print("Спасибо за использование нашего киоска! До свидания.")
                break
            else:
                print("Некорректный ввод. Пожалуйста, выберите пункт из меню.")

    def _print_main_menu(self): 
        print("\n--- Главное Меню Киоска ---")
        print("1. Добавить хот-дог в текущий заказ")
        print("2. Посмотреть текущий заказ")
        print("3. Оформить и оплатить заказ")
        print("4. Посмотреть остатки ингредиентов (инвентарь)")
        print("5. Посмотреть отчет о продажах")
        print("6. Посмотреть, какие ингредиенты нужно закупить")
        print("0. Выйти из приложения")

    def _handle_add_hotdog_to_order(self): 
        print("\n--- Добавление хот-дога в заказ ---")
        print("1. Выбрать хот-дог по стандартному рецепту")
        print("2. Создать свой (кастомный) хот-дог")
        print("0. Вернуться в главное меню")
        sub_choice = input("Ваш выбор: ").strip()

        if sub_choice == '1':
            recipes = self.kiosk.get_available_standard_recipes()
            if not recipes:
                print("Извините, стандартные рецепты временно недоступны.")
                return
            print("\nДоступные стандартные рецепты хот-догов:")
            for i, recipe in enumerate(recipes):
                print(f"{i+1}. {recipe.name} - Цена: ${recipe.fixed_price:.2f}")
            try:
                recipe_idx_choice = int(input("Выберите номер рецепта: ")) - 1
                if 0 <= recipe_idx_choice < len(recipes):
                    selected_recipe_name = recipes[recipe_idx_choice].name
                    self.kiosk.add_standard_hotdog_to_order(selected_recipe_name)
                else:
                    print("Неверный номер рецепта.")
            except ValueError:
                print("Некорректный ввод. Пожалуйста, введите число.")
        elif sub_choice == '2':
            self._handle_create_custom_hotdog()
        elif sub_choice == '0':
            return
        else:
            print("Некорректный выбор. Пожалуйста, попробуйте снова.")

    def _select_ingredient_from_category_for_custom(self, category_name: str, prompt_message: str, allow_skip: bool = False) -> Optional[Ingredient]: 
        """Помощник для выбора ингредиента определенной категории для кастомного хот-дога."""
        print(f"\n--- {prompt_message} ---")
        available_ingredients = self.inventory.get_available_ingredients_by_category(category_name)
        
        if not available_ingredients:
            print(f"К сожалению, ингредиенты из категории '{category_name}' закончились.")
            return None
        
        print(f"Доступные '{category_name}':")
        for i, ing in enumerate(available_ingredients):
            print(f"{i+1}. {ing.name} (Доп. цена: ${ing.price_to_customer:.2f})")
        if allow_skip:
            print("0. Пропустить выбор / Закончить добавление из этой категории")
        
        while True:
            try:
                user_choice = int(input(f"Выберите номер '{category_name}' (или 0 для пропуска, если доступно): "))
                if allow_skip and user_choice == 0:
                    return None
                
                selected_idx = user_choice - 1
                if 0 <= selected_idx < len(available_ingredients):
                    chosen_ingredient = available_ingredients[selected_idx]
                    # Дополнительная проверка на случай, если ингредиент закончился между листингом и выбором
                    if self.inventory.get_stock_quantity(chosen_ingredient.name) > 0:
                        return chosen_ingredient
                    else:
                        print(f"Извините, ингредиент '{chosen_ingredient.name}' только что закончился. Пожалуйста, выберите другой.")
                        # Повторно отображаем список, так как он мог измениться
                        return self._select_ingredient_from_category_for_custom(category_name, prompt_message, allow_skip)
                else:
                    print("Неверный номер выбора. Попробуйте еще раз.")
            except ValueError:
                print("Некорректный ввод. Пожалуйста, введите число.")

    def _handle_create_custom_hotdog(self): 
        print("\n--- Создание Кастомного Хот-дога ---")
        builder = CustomHotDogBuilder() 
        custom_hotdog_name = input("Введите название для вашего кастомного хот-дога (например, 'Мега Хот-дог'): ")
        builder.reset(custom_hotdog_name if custom_hotdog_name else "Мой Кастомный Хот-дог")

        # Список для отслеживания ингредиентов, которые были "взяты" со склада для этого хот-дога
        # чтобы можно было их вернуть, если пользователь отменит создание
        ingredients_used_for_this_custom_hotdog: List[Ingredient] = []

        # 1. Выбор Булочки (обязательно)
        bun = self._select_ingredient_from_category_for_custom("булочка", "Выберите основу: Булочка")
        if not bun: 
            print("Без булочки хот-дог не получится. Создание отменено.")
            return
        if not self.inventory.use_ingredient(bun.name): # Списываем со склада
            print(f"Не удалось использовать булочку '{bun.name}'. Создание отменено.")
            return 
        builder.add_base_ingredient(bun)
        ingredients_used_for_this_custom_hotdog.append(bun)

        # 2. Выбор Сосиски (обязательно)
        sausage = self._select_ingredient_from_category_for_custom("сосиска", "Выберите основу: Сосиска")
        if not sausage: 
            print("Без сосиски хот-дог не получится. Создание отменено.")
            self.inventory.add_ingredient_to_stock(bun, 1) # Возвращаем булочку на склад
            return
        if not self.inventory.use_ingredient(sausage.name): # Списываем
            print(f"Не удалось использовать сосиску '{sausage.name}'. Создание отменено.")
            self.inventory.add_ingredient_to_stock(bun, 1) # Возвращаем булочку
            return
        builder.add_base_ingredient(sausage)
        ingredients_used_for_this_custom_hotdog.append(sausage)
        
        # 3. Выбор Соусов (опционально, можно несколько)
        print("\n--- Добавление Соусов (выбирайте по одному, введите 0 для завершения) ---")
        while True:
            sauce = self._select_ingredient_from_category_for_custom("соус", "Выберите Соус", allow_skip=True)
            if not sauce: break # Пользователь решил закончить выбор соусов
            if self.inventory.use_ingredient(sauce.name): # Списываем
                builder.add_sauce(sauce)
                ingredients_used_for_this_custom_hotdog.append(sauce)
                print(f"Соус '{sauce.name}' добавлен.")
            else:
                print(f"Не удалось добавить соус '{sauce.name}' (возможно, закончился).")

        # 4. Выбор Топпингов (опционально, можно несколько)
        print("\n--- Добавление Топпингов (выбирайте по одному, введите 0 для завершения) ---")
        while True:
            topping = self._select_ingredient_from_category_for_custom("топпинг", "Выберите Топпинг", allow_skip=True)
            if not topping: break # Пользователь решил закончить выбор топпингов
            if self.inventory.use_ingredient(topping.name): # Списываем
                builder.add_topping(topping)
                ingredients_used_for_this_custom_hotdog.append(topping)
                print(f"Топпинг '{topping.name}' добавлен.")
            else:
                print(f"Не удалось добавить топпинг '{topping.name}' (возможно, закончился).")
        
        final_custom_hotdog = builder.get_hotdog()
        if final_custom_hotdog:
            # Цена кастомного хот-дога формируется из суммы цен добавленных ингредиентов
            self.kiosk.add_custom_hotdog_to_order(final_custom_hotdog)
        else: 
            print("Произошла ошибка при создании кастомного хот-дога. Ингредиенты будут возвращены на склад.")
            for ing in ingredients_used_for_this_custom_hotdog: # Возвращаем все "взятые" ингредиенты
                self.inventory.add_ingredient_to_stock(ing, 1)

    def _handle_finalize_and_pay_order(self): 
        if not self.kiosk.current_order_hotdogs:
            print("Ваш заказ пуст. Пожалуйста, сначала добавьте хот-доги в заказ.")
            return

        self.kiosk.display_current_order() # Показываем заказ перед оплатой
        print("\n--- Выбор Способа Оплаты ---")
        
        # Получаем доступные методы оплаты из киоска
        payment_options_map = self.kiosk.payment_strategies 
        payment_keys_list = list(payment_options_map.keys())

        if not payment_keys_list:
            print("Извините, способы оплаты временно недоступны.")
            return

        for i, key in enumerate(payment_keys_list):
            print(f"{i+1}. {payment_options_map[key].get_payment_type_name()}")
        
        try:
            payment_choice_idx = int(input("Выберите номер способа оплаты: ")) - 1
            if 0 <= payment_choice_idx < len(payment_keys_list):
                selected_payment_key = payment_keys_list[payment_choice_idx]
                self.kiosk.finalize_and_pay_order(selected_payment_key)
            else:
                print("Неверный номер способа оплаты.")
        except ValueError:
            print("Некорректный ввод. Пожалуйста, введите число.")

Файл: main_kiosk_app.py (Основной файл приложения, точка входа)

In [19]:
# main_kiosk_app.py
# Импорт всех необходимых модулей и классов


def setup_initial_kiosk_inventory(manager: InventoryManager): 
    """Наполняет инвентарь начальными данными, если он пуст."""
    # Булочки
    manager.add_ingredient_to_stock(Ingredient("Классическая булочка", 0.30, 0.75, "булочка"), 50)
    manager.add_ingredient_to_stock(Ingredient("Булочка с кунжутом", 0.35, 0.85, "булочка"), 30)
    manager.add_ingredient_to_stock(Ingredient("Бриошь булочка", 0.45, 1.00, "булочка"), 20)
    # Сосиски
    manager.add_ingredient_to_stock(Ingredient("Говяжья сосиска 'Классика'", 0.80, 1.50, "сосиска"), 50)
    manager.add_ingredient_to_stock(Ingredient("Острый чоризо 'Огонек'", 1.00, 1.75, "сосиска"), 30)
    manager.add_ingredient_to_stock(Ingredient("Вегетарианская сосиска 'Здоровье'", 0.90, 1.60, "сосиска"), 30)
    # Соусы
    manager.add_ingredient_to_stock(Ingredient("Кетчуп томатный", 0.05, 0.25, "соус"), 100)
    manager.add_ingredient_to_stock(Ingredient("Горчица дижонская", 0.06, 0.30, "соус"), 100)
    manager.add_ingredient_to_stock(Ingredient("Майонез провансаль", 0.07, 0.30, "соус"), 80)
    manager.add_ingredient_to_stock(Ingredient("Соус Чили острый", 0.08, 0.35, "соус"), 60)
    manager.add_ingredient_to_stock(Ingredient("Сырный соус", 0.15, 0.50, "соус"), 50)
    # Топпинги
    manager.add_ingredient_to_stock(Ingredient("Сладкий луковый релиш", 0.10, 0.40, "топпинг"), 50)
    manager.add_ingredient_to_stock(Ingredient("Перчики Халапеньо (острые)", 0.12, 0.45, "топпинг"), 40)
    manager.add_ingredient_to_stock(Ingredient("Маринованные огурчики", 0.10, 0.40, "топпинг"), 50)
    manager.add_ingredient_to_stock(Ingredient("Хрустящий жареный лук", 0.15, 0.50, "топпинг"), 40)
    manager.add_ingredient_to_stock(Ingredient("Тертый сыр Чеддер", 0.20, 0.60, "топпинг"), 40)
    print("Первоначальная настройка инвентаря киоска завершена.")

def run_application():
    """Основная функция для запуска приложения киоска."""
    
    # 1. Инициализация Менеджера Инвентаря (Одиночка)
    # Файл для хранения состояния инвентаря
    inventory_file_name = "hotdog_kiosk_inventory_data.json" 
    inventory_manager = InventoryManager(inventory_file=inventory_file_name, low_stock_threshold=10)
    
    # 2. Настройка Наблюдателя за низким запасом ингредиентов
    console_stock_notifier = ConsoleLowStockNotifier()
    inventory_manager.add_observer(console_stock_notifier)

    # 3. Проверка и возможное наполнение инвентаря при первом запуске
    # Доступ к _stock здесь только для проверки инициализации, в остальном через публичные методы
    if not inventory_manager._stock: 
        print(f"Инвентарь в файле '{inventory_file_name}' пуст или файл отсутствует. Выполняется первоначальное наполнение...")
        setup_initial_kiosk_inventory(inventory_manager)
    else:
        print(f"Инвентарь успешно загружен из файла '{inventory_file_name}'.")

    # 4. Определение Стандартных Рецептов Хот-догов
    standard_recipes_list = [
        HotDogRecipe("Классический Американский", ["Классическая булочка", "Говяжья сосиска 'Классика'", "Кетчуп томатный", "Горчица дижонская"], 4.50),
        HotDogRecipe("Острый Дьявол", ["Бриошь булочка", "Острый чоризо 'Огонек'", "Перчики Халапеньо (острые)", "Соус Чили острый"], 5.75),
        HotDogRecipe("Вегетарианское Удовольствие", ["Булочка с кунжутом", "Вегетарианская сосиска 'Здоровье'", "Сладкий луковый релиш", "Кетчуп томатный"], 5.25),
        HotDogRecipe("Сырный Бум", ["Классическая булочка", "Говяжья сосиска 'Классика'", "Сырный соус", "Тертый сыр Чеддер", "Хрустящий жареный лук"], 6.00)
    ]

    # 5. Настройка Строителя Хот-догов
    hotdog_builder_instance = CustomHotDogBuilder()

    # 6. Настройка Системы Сохранения Заказов
    # Файл для истории всех заказов
    orders_history_file_name = "all_hotdog_orders_history.jsonl" 
    order_persistence_system = FileOrderPersistence(filename=orders_history_file_name)

    # 7. Настройка Стратегии Скидок (активная стратегия)
    # Скидка 10% при заказе от 3-х хот-догов
    active_discount_strategy = QuantityDiscountStrategy(min_quantity=3, discount_percentage=0.10) 

    # 8. Настройка доступных Стратегий Оплаты
    available_payment_strategies = {
        "наличные": CashPaymentStrategy(),
        "карта": CardPaymentStrategy()
    }

    # 9. Создание и Настройка Фасада Киоска
    kiosk_application_facade = HotDogKiosk(
        inventory_manager=inventory_manager,
        hotdog_builder=hotdog_builder_instance,
        order_persistence=order_persistence_system,
        discount_strategy=active_discount_strategy,
        payment_strategies=available_payment_strategies,
        standard_recipes=standard_recipes_list
    )

    # 10. Создание и Запуск Пользовательского Интерфейса
    console_user_interface = ConsoleUI(kiosk_application_facade, inventory_manager)
    console_user_interface.run()

if __name__ == "__main__":
    run_application()

Инвентарь успешно загружен из файла 'hotdog_kiosk_inventory_data.json'.
Добро пожаловать в автоматизированный киоск по продаже хот-догов!

Начат новый заказ. Корзина очищена.

--- Главное Меню Киоска ---
1. Добавить хот-дог в текущий заказ
2. Посмотреть текущий заказ
3. Оформить и оплатить заказ
4. Посмотреть остатки ингредиентов (инвентарь)
5. Посмотреть отчет о продажах
6. Посмотреть, какие ингредиенты нужно закупить
0. Выйти из приложения


Текущий заказ пуст.

--- Главное Меню Киоска ---
1. Добавить хот-дог в текущий заказ
2. Посмотреть текущий заказ
3. Оформить и оплатить заказ
4. Посмотреть остатки ингредиентов (инвентарь)
5. Посмотреть отчет о продажах
6. Посмотреть, какие ингредиенты нужно закупить
0. Выйти из приложения

--- Добавление хот-дога в заказ ---
1. Выбрать хот-дог по стандартному рецепту
2. Создать свой (кастомный) хот-дог
0. Вернуться в главное меню

Доступные стандартные рецепты хот-догов:
1. Классический Американский - Цена: $4.50
2. Острый Дьявол - Цена: $5.75
3. Вегетарианское Удовольствие - Цена: $5.25
4. Сырный Бум - Цена: $6.00
Хот-дог по рецепту 'Вегетарианское Удовольствие' успешно добавлен в текущий заказ.
Хот-дог: Вегетарианское Удовольствие
  Ингредиенты: Булочка с кунжутом, Вегетарианская сосиска 'Здоровье', Кетчуп томатный, Сладкий луковый релиш
  Себестоимость для киоска: $1.40
  Цена для клиента: $5.25

--- Главное Меню Киоска ---
1. Добавить хот-дог в текущий заказ
2. Посмотреть текущий 